# Routing 
- Trên thực tế, dữ liệu có thể được lưu trữ ở nhiều nguồn khác nhau với các nguồn dữ liệu chứa các thông tin riêng biệt. Việc thiết kế hệ thống RAG nhắm query tới đúng nguồn dữ liệu hay phân loại các câu query phù hợp với từng dạng prompt, cần phải có một module phần loại, phân chia được gọi là routing. Dựa và 2 cách thức sử dụng ở trên mà người ta có thể phân loại thành 2 cách thức là Logical routing (phân loại để lựa chọn nguồn dữ liệu phù hợp) và Semantic routing (phân loại để lấy câu prompt thích hợp )

<p align="center">
    <img src="../doc/image/routing.png" alt="basic-pipeline" width="600"/>
</p>

- Bản chất router giống như một câu lệnh if/else giúp câu query được lựa chọn vào luồng phù hợp. Các cách thực hiện router có thể biểu diễn dưới hình sau: 


<p align="center">
    <img src="../doc/image/routers.png" alt="basic-pipeline" width="600"/>
</p>


# Setup 

In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

# Phân loại dựa theo cách thức 
Dựa và 2 cách thức sử dụng ở trên mà người ta có thể phân loại thành 2 cách thức là Logical routing (phân loại để lựa chọn nguồn dữ liệu phù hợp) và Sematic routing (phân loại để lấy câu prompt thích hợp )

## Logical routing 
- Là phương pháp lựa chọn các nguồn dữ liệu phù hợp dựa vào query của người dùng


<p align="center">
    <img src="../doc/image/logical-routing.png" alt="basic-pipeline" width="600"/>
</p>

In [7]:
# xây dựng các retrievers có sẵn 
from typing import Literal 
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI 
from langchain_google_vertexai import ChatVertexAI


# xây dựng base-model minh hoạ cho các cơ sở dữ liệu 
class RouteQuery(BaseModel): 
    """ Router để người dùng tìm kiếm source phù hợp """

    datasource : Literal["python", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )


# xây dựng LLm với function calling, đảm bảo dữ liệu đầu ra 
llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-flash-001', temperature = 0, api_key=GOOGLE_API_KEY)
structured_llm = llm.with_structured_output(RouteQuery)

# xây dựng prompt phân loại 
template = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template), 
        ("human", "{question}"),
    ]
)

# xây dựng chain 
router_chain = prompt | structured_llm 

In [8]:
# ví dụ minh họa 
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human"])
prompt.invoke("french")
"""

result = router_chain.invoke({"question": question})

In [9]:
result

RouteQuery(datasource='python')

## Semantic Router
- Thực tế, với tùy vào input của người dùng mà các câu prompt có thể được thiết kế khác nhau để tối ưu việc trích xuất thông tin. Vấn đề đặt ra là làm sao để biết query của người dùng phù hợp với prompt nào, người ta đã giới thiệu phương pháp router đề giải quyết vấn đề trên. 


<p align="center">
    <img src="../doc/image/semantic-routing.png" alt="basic-pipeline" width="600"/>
</p>

- Thông qua việc embedding cả prompt lẫn query từ người dùng, sử dụng các thuật toán tính độ tương đồng, có thể lựa chọn ra prompt gần nhất với câu hỏi của người đùng. 

In [12]:
from langchain.utils.math import cosine_similarity 
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.prompts import PromptTemplate 
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI


# tạo tập prompt bất kì 
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""


# embedding prompt 
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001",  google_api_key = GOOGLE_API_KEY)
prompt_templates = [physics_template, math_template]
prompt_embedding = embedding.embed_documents(prompt_templates)



# lựa chọn prompt thông quan qua query đã được embedding 
def prompt_router(input): 
    # embedding query 
    query_embedding = embedding.embed_query(input["query"])
    # tính toán similarity 
    similarity = cosine_similarity([query_embedding], prompt_embedding)[0]
    most_similarity = prompt_templates[similarity.argmax()]

    return PromptTemplate.from_template(most_similarity)


# khởi tạo chain 
semantic_chain = (
    {"query" : RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm 
    | StrOutputParser()
)


# trích xuất kết quả 
semantic_chain.invoke("What's a black hole")

"A black hole is a region of spacetime where gravity is so strong that nothing, not even light, can escape. It's formed when a massive star collapses at the end of its life. \n\nImagine a giant star, much bigger than our sun, running out of fuel. Without the outward pressure from nuclear fusion, gravity takes over, crushing the star's core into an incredibly dense point called a singularity.  \n\nThe singularity's gravity is so intense that it warps the fabric of spacetime around it, creating a region called the event horizon.  Anything that crosses the event horizon is trapped forever, doomed to spiral into the singularity. \n\nThink of it like a cosmic drain, pulling everything in, but with no way out. \n\nWhile we can't directly see black holes, we can observe their effects on surrounding matter and light.  They're fascinating objects that continue to challenge our understanding of gravity and the universe. \n"